In [ ]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

from plotly import express as px
from typing import List, Tuple
import pickle
import numpy as np

In [ ]:
import sys
sys.path.append('../../../../')

## load

In [ ]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training/03_04_with_pesto_and_coord/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

In [ ]:
fold = folds_training_dicts[0]
fold.keys()

In [ ]:
input_data = fold['components_train']
coordinates = fold['coordinates_train']
size_value = fold['sizes_train']
n_patches_hot_encoded_value = fold['num_patches_train']
max_number_of_patches = 10

## debug pairwise transition

In [ ]:
from model import build_model, create_masked_inputs

In [ ]:
model = build_model(features_mlp_hidden_sizes=[[32, 32]],
            features_mlp_dropout_rate=0.2,
            output_mlp_hidden_sizes=[[32, 32]], output_mlp_dropout_rate=0.2,
            attention_mlp_hidden_sizes=[[32, 32]], attention_mlp_dropout_rate=0.2,
            activation='relu', input_shape=(10, 9), max_number_of_patches=10, attention_dimension=32,
            gaussian_xrange=(0, 100),
            pairs_channel_dimension=32, num_heads=8, use_pair_bias=True)

In [ ]:
model.summary()

In [ ]:
batch_size = 7

In [ ]:
model([input_data[:batch_size], coordinates[:batch_size], size_value[:batch_size], n_patches_hot_encoded_value[:batch_size]])

In [ ]:
# model.save(f'tmp_model.keras')

In [ ]:
# model.load_weights(f'tmp_model.keras')

In [ ]:
from models.patch_to_score.models.modules.gaussian_kernel import GaussianKernel, initialize_gaussian_kernel_uniform, initialize_GaussianKernel

In [ ]:
from model import create_masked_inputs

features, pairwise_distances = create_masked_inputs(
    input_data, coordinates, size_value, n_patches_hot_encoded_value, max_number_of_patches)

In [ ]:
pairwise_distances.shape

In [ ]:
flat_distances = pairwise_distances[pairwise_distances != 0].numpy()

In [ ]:
initial_values = initialize_GaussianKernel(np.expand_dims(flat_distances, 1), 8, n_init=4)

In [ ]:
centers, stds = initial_values

In [ ]:
fitted_gaussian_kernel = GaussianKernel(8, initial_values, 'diag')

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.stats import norm

x = np.linspace(flat_distances.min() - 3, flat_distances.max() + 3, 5000)

# Plot data histogram
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=flat_distances[::10], 
    histnorm="probability density",
    name="Data",
    opacity=0.5,
    nbinsx=10_000
))

print('here')
# Plot each Gaussian
for mean, std in zip(centers[0], stds[0]):
    y = norm.pdf(x, loc=mean, scale=std)
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode="lines",
        name=f"N({mean}, {std**2:.2f})"
    ))

# Layout
fig.update_layout(
    title="Data and Gaussian Distributions",
    xaxis_title="x",
    yaxis_title="Density"
)

fig.show()


In [ ]:
# px.histogram(flat_distances[flat_distances < 50][::2], nbins=10_000)